Autor: so jae woo    
Data src :  qesg news / 28 cat / 본문 포함.    
Summary : KLUE기반의 버트 사정 임베딩 모델 사용.   

참고 : https://bo-10000.tistory.com/154


In [12]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import transformers
transformers.__version__ # 
from sklearn.model_selection import StratifiedKFold

from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import TextClassificationPipeline
from sklearn.utils import class_weight

from lib_offline import prep_mecab_noun_offline
from lib2 import text_preprocessing
import pickle
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset

# 데이터 준비 
## QESG 데이터 준비 

In [2]:
df_qesg = pd.read_parquet("./df_qesg28_final.pq")
df_qesg = df_qesg.loc[df_qesg.y.notna()]
df_qesg = df_qesg.loc[df_qesg.y != '기타']
df_qesg = df_qesg.loc[df_qesg.y != '제외']

In [3]:
len(pd.unique(df_qesg.y))

28

### 데이터 클리닝

### 짧은 본문 제거

In [4]:
def check_contents_clean(row):
    c = row.contents
    cc = row.contents_clean
    
    if len(c) == 0:
        return False
    
    if len(cc)/len(c)> 0.7:
        return True
    else:
        return False

In [5]:
bol_list = [ check_contents_clean(row) for i , row in df_qesg.iterrows() ]
df_qesg['size'] = df_qesg[bol_list].contents_clean.apply(lambda x : len(x))
df_qesg = df_qesg.loc[df_qesg['size'] > 40]

## 라벨 숫자 인코딩
lbl_name = sorted(df_qesg['y'].unique().tolist())
lbl_num = list(range(len(lbl_name)))
lbl_name2num = dict(zip(lbl_name,lbl_num))
lbl_num2name = dict(zip(lbl_num,lbl_name))
df_qesg['y2num']  = df_qesg.y.apply(lambda x : lbl_name2num[x])

# BERT 임베딩 

In [6]:
from transformers import TrainingArguments
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import Trainer


def tokenize_function(x_src):
    return tokenizer(x_src, truncation=True, padding=True)


model = AutoModel.from_pretrained("klue/bert-base")
max_seq_len = 2028
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base", truncation=True, max_seq_len=max_seq_len)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
%%time
#X = np.array(df_qesg['contents_clean'].tolist())
#y = np.array(df_qesg['y2num'].tolist())

X = np.array(df_qesg['contents_clean'].tolist()[:100])
y = np.array(df_qesg['y2num'].tolist()[:100])

X_train = tokenizer(X.tolist(), truncation=True, padding=True)

CPU times: total: 328 ms
Wall time: 243 ms


In [26]:
class CustomDataset(Dataset):
    def __init__(self, x,y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return {
        'input_ids' : self.x['input_ids'][idx],
        'token_type_ids' : self.x['token_type_ids'][idx],
        'attention_mask' : self.x['attention_mask'][idx]
        }
        

In [27]:
custom_data = CustomDataset(X_train,y)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [28]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1, 
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1,               # How often to print logs
    do_train=True,                   # Perform training
    fp16=True,                       # Use mixed precision
    fp16_opt_level="02",             # mixed precision mode
    run_name="test1",       # experiment name
    seed=2023                           # Seed for experiment reproducibility 3x3
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=custom_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [31]:
len(y)

100

In [29]:
trainer.train()

***** Running training *****
  Num examples = 3
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3


KeyError: 'loss'

In [6]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [7]:
# load tokenizer 
max_seq_len = 2028
tokenizer = BertTokenizer.from_pretrained('klue/bert-base', truncation=True, max_seq_len=max_seq_len)

In [8]:
## 라벨 숫자 인코딩
lbl_name = sorted(df_qesg['y'].unique().tolist())
lbl_num = list(range(len(lbl_name)))
lbl_name2num = dict(zip(lbl_name,lbl_num))
lbl_num2name = dict(zip(lbl_num,lbl_name))
df_qesg['y2num']  = df_qesg.y.apply(lambda x : lbl_name2num[x])

# 트레이닝

In [9]:
X = np.array(df_qesg['contents_clean'].tolist())
y = np.array(df_qesg['y2num'].tolist())

In [10]:
%%time
# 원큐
class_weights = class_weight.compute_class_weight(class_weight  = 'balanced',classes= np.unique(y), y = y) 
class_weights_dict = dict(zip(  list(range(len(class_weights))),class_weights))

X_train = tokenizer(X.tolist(), truncation=True, padding=True)


train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    y
))

In [ ]:
cb_save = tf.keras.callbacks.ModelCheckpoint(
    './model',
    monitor = 'accuracy',
  
    save_best_only = True,
    save_weights_only = False,
    mode= 'auto',
    save_freq='epoch',

)

In [ ]:
%%time
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=df_qesg['y2num'].nunique(), from_pt=True)
loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

history =model.fit(
train_dataset.shuffle(1000).batch(16), epochs=10, batch_size=32,class_weight= class_weights_dict,)

In [ ]:
model.save_weights('model_weight_28_embedding', save_format='tf')

## 임베딩 결과 저장하기 

In [ ]:
%%time
X_train = tokenizer(X.tolist(), truncation=True, padding=True)
del tokenizer
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=df_qesg['y2num'].nunique(), from_pt=True)
model.load_weights('model_weight_28_embedding')

In [14]:
del tokenizer
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=df_qesg['y2num'].nunique(), from_pt=True)
model.load_weights('model_weight_28_embedding')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
%%time
res_emb = model.bert.embeddings(X_train['input_ids'],X_train['attention_mask'],X_train['token_type_ids'])

Unexpected exception formatting exception. Falling back to standard exception


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
del tokenizer

In [12]:
type(X_train)

transformers.tokenization_utils_base.BatchEncoding